In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 11:14:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=transformations_actions>

# RDD 생성
파이썬의 리스트를 이용해서 RDD 생성 = `parallelize` (병렬처리)

In [3]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

# Actions
- 미뤄왓던 Transformations 작업을 모두 실행학, 그 결과를 반환하는 작업
- RDD 생성이 아닌, 실제 데이터를 출력하는 역할을 한다.

In [4]:
# 각 원소 별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 3,
             '짬뽕': 3,
             '마라탕': 2,
             '떡볶이': 1,
             '쌀국수': 2,
             '라면': 2,
             '우동': 1})

In [5]:
# 상위 n개의 데이터 가져오기
foods.take(5)

['짜장면', '짬뽕', '마라탕', '떡볶이', '쌀국수']

In [6]:
# 전체 데이터 가져오기
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

In [7]:
# RDD 크기 구하기
foods.count()

14

# Transformations
## Narrow Transformations
- 1:1 변환을 의미한다.
- 하나의 열을 조작하기 위해서 다른 열이나 파티션의 데이터를 사용하지 않는다.

In [8]:
# map(<task>)
# RDD 내의 데이터를 하나씩 꺼내서 `<task>`가 적용된 새로운 RDD가 만들어진다.
simple_rdd = sc.parallelize([1,2,3])
simple_rdd

ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD.scala:274

In [9]:
# task 함수를 정의할 때 "반드시" 리턴이 있어야 한다. 없으면 None이 나옴!!
simple_rdd2 = simple_rdd.map(lambda x : x * 2) # lambda식은 return이 있는데 생략되어있는것임. lambda x : return(x * 2)
simple_rdd2

PythonRDD[5] at RDD at PythonRDD.scala:53

In [10]:
simple_rdd2.collect()

[2, 4, 6]

### `flatMap(<task>)`
- flat -> 1차원배열을 의미
- map 함수와 매우 흡사하나, flatMap함수는 map의 모든 결과를 1차원 배열 형식으로 

In [13]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [14]:
moviesRDD = sc.parallelize(movies)
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [15]:
mapMovies = moviesRDD.map(lambda x : x.split()) # x = 영화제목
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [16]:
flatMapMovies = moviesRDD.flatMap(lambda x : x.split())
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

### `filter(<task>)`
- `task`의 결과가 True인 데이터만 추출

In [19]:
filtered_movies = flatMapMovies.filter(lambda x : x != '매트릭스')
filtered_movies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

## Wide Transformations
- 데이터의 파티션 또는 노드가 바뀔 수 있는 변환
- 셔플링이 일어날 수 있기 때문에 느리다.

In [21]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수", "짬뽕밥", "짬짜면", "볶음밥", "볶짬면"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수',
 '짬뽕밥',
 '짬짜면',
 '볶음밥',
 '볶짬면']

### `groupBy(<task>)`
- groupBy의 task는 데이터를 묶어줄 수 있는 기준을 설정 (map:변환 task)

In [23]:
# 그룹핑의 기준을 각 원소의 첫 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[21] at RDD at PythonRDD.scala:53

In [24]:
result = foodsGroup.collect()
result  # (앞글자 = key, 그룹 = value)

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f1d4c4d03a0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f1d4c202a90>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f1d4c202be0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f1d4c2025e0>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f1d4c202520>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f1d4c330250>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f1d4c47a400>),
 ('볶', <pyspark.resultiterable.ResultIterable at 0x7f1d4c47a940>)]

In [25]:
for k,v in result:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕', '짬뽕', '짬뽕밥', '짬짜면']
마 ['마라탕', '마라탕']
떡 ['떡볶이']
쌀 ['쌀국수', '쌀국수']
라 ['라면', '라면']
우 ['우동']
볶 ['볶음밥', '볶짬면']


In [26]:
sc.stop()